In [8]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [9]:
# 1. Carregar a planilha
df = pd.read_excel("Itens_Agrupados_com_Dia_Semana.xlsx")

In [10]:
# 2. Converter data e extrair features
df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%y')
df['Dia da Semana'] = df['Data'].dt.dayofweek
df['Mês'] = df['Data'].dt.month


In [11]:
# 3. Pivotar: cada linha é um dia, cada coluna é um item
df_pivot = df.pivot_table(index='Data', columns='Descrição', values='Qtde', fill_value=0)

In [12]:
# 4. Adicionar features ao DataFrame de entrada
df_features = df[['Data', 'Dia da Semana', 'Mês']].drop_duplicates().set_index('Data')
df_final = df_features.join(df_pivot)

In [13]:
# 5. Separar X e Y
X = df_final[['Dia da Semana', 'Mês']]
Y = df_final.drop(columns=['Dia da Semana', 'Mês'])

In [14]:
# 6. Dividir em treino e teste (80/20) sem embaralhar (importante para séries temporais)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)

print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)


X_train shape: (144, 2)
Y_train shape: (144, 38)


In [15]:
# 7. Criar e treinar o modelo de Regressão Linear para múltiplos itens
modelo_linear = MultiOutputRegressor(LinearRegression())
modelo_linear.fit(X_train, Y_train)


MultiOutputRegressor(estimator=LinearRegression())

In [16]:
# 8. Fazer previsões com o modelo
Y_pred = modelo_linear.predict(X_test)


In [17]:
# 9. Avaliar o modelo com métricas padrão de regressão
r2 = r2_score(Y_test, Y_pred)
mae = mean_absolute_error(Y_test, Y_pred)
rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))

print("Avaliação da Regressão Linear:")
print(f"R² Score médio: {r2:.4f}")
print(f"MAE médio:      {mae:.4f}")
print(f"RMSE médio:     {rmse:.4f}")


Avaliação da Regressão Linear:
R² Score médio: -0.3333
MAE médio:      1.5646
RMSE médio:     2.4386


In [18]:
import json

# Criar o dicionário com os resultados
linear_results = {
    "Modelo": "Regressão Linear",
    "Tipo": "Regressão",
    "R² Score": r2,
    "MAE": mae,
    "RMSE": rmse
}

# Salvar em arquivo JSON
with open("resultado_regressao_linear.json", "w") as f:
    json.dump(linear_results, f)

print("Resultado da Regressão Linear salvo como 'resultado_regressao_linear.json'")


Resultado da Regressão Linear salvo como 'resultado_regressao_linear.json'
